<a href="https://colab.research.google.com/github/arif5535/Semantic-segmentation/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.9.1 google-auth-oauthlib pandas-gbq

In [ ]:
from tensorflow import keras

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models

In [ ]:
import segmentation_models as sm

In [ ]:
!pip install matplotlib

In [ ]:
import tensorflow as tf
import glob
import cv2
import os
import numpy as np

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

In [ ]:
#Resizing images is optional, CNNs are ok with large images
SIZE_X = 256 #Resize images (height  = X, width = Y)
SIZE_Y = 256

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
#Capture training image info as a list
train_images = []
for directory_path in glob.glob("/content/gdrive/MyDrive/Segmentation/Images"):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        #print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        # img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        # print(train_images)
        #train_labels.append(label)
#Convert list to array for machine learning processing
train_images = np.array(train_images)

In [ ]:
#Capture mask/label info as a list
train_masks = []
for directory_path in glob.glob("/content/gdrive/MyDrive/Segmentation/Masks"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        mask = cv2.imread(mask_path, 0)
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X))
        # mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR)
        train_masks.append(mask)
        #train_labels.append(label)
#Convert list to array for machine learning processing
train_masks = np.array(train_masks)

In [ ]:
#Use customary x_train and y_train variables
X = train_images
Y = train_masks

Y = np.expand_dims(Y, axis=3) #May not be necessary.. leftover from previous code

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# preprocess input
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)

In [ ]:
# define model
from tensorflow.keras.utils import get_file

In [ ]:
model = sm.Unet(BACKBONE, encoder_weights='imagenet')
model.compile(optimizer='Adam', loss=sm.losses.bce_jaccard_loss, metrics=[sm.metrics.iou_score])

print(model.summary())

In [ ]:
#type_cast_float_32

y_train = tf.cast(y_train, tf.float32)
y_val = tf.cast(y_val, tf.float32)

In [ ]:
#train model

history=model.fit(x = x_train,
          y = y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          validation_data=(x_val, y_val))

In [ ]:
accuracy = model.evaluate(x_val, y_val)
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
model.save('/content/gdrive/MyDrive/Segmentation/model.h5')

In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/gdrive/MyDrive/Segmentation/model.h5', compile=False)

In [ ]:
#Test on a different image
#READ EXTERNAL IMAGE...
test_img = cv2.imread('/content/gdrive/MyDrive/Segmentation/Images/401.jpg', cv2.IMREAD_COLOR)
test_img = cv2.resize(test_img, (SIZE_Y, SIZE_X))
test_img = cv2.cvtColor(test_img, cv2.COLOR_RGB2BGR)
test_img = np.expand_dims(test_img, axis=0)
#plt.imshow(test_img,cmap = 'gray')
prediction = model.predict(test_img)

In [ ]:
#View and Save segmented image
prediction_image = prediction.reshape(mask.shape)
plt.imshow(prediction_image, cmap='gray')
plt.imsave('/content/gdrive/MyDrive/Segmentation/test0_segmented.jpg', prediction_image, cmap='gray')